In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

print(sys.path)

['/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/scur0279/MMA/.venv/lib64/python3.9/site-packages', '/home/scur0279/MMA/.venv/lib/python3.9/site-packages', '/gpfs/home5/scur0279/MMA/app']


In [3]:
from db.database import Database
import uuid

/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scur0279/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/scur0279/MMA/.venv/lib64/python3.9/site-packages/lpips/weights/v0.1/alex.pth


In [3]:
ls

create.py    example.ipynb       images/      mma.db
database.py  gilian_database.py  __init__.py  __pycache__/


#### stuff for testing the saving

In [4]:
from modules.prompt import Prompt
from modules.prompt_image import PromptImage
from PIL import Image

# Step 1: Reset DB and insert users/chats (keep as is)
datab = Database()
datab.connect()
datab.reset_tables()

# --- Step 1: Create Users and Create Chats ---
datab.insert_user("first_test_user", "password123")
datab.insert_user("second_test_user", "password456")

datab.insert_chat("First Test Chat", 1)
datab.insert_chat("Second Test Chat", 2)

chat_id_1=1
chat_id_2=2

# Step 2: Generate Prompt instances
# Prompy inputs are: prompt: str, depth: int, input_image: PromptImage = None, suggestion_used: str = None, modified_suggestion: bool = False):
prompt_1 = Prompt(prompt="Generate a bottle of perfume", depth=1)
prompt_2 = Prompt(prompt="Generate a bottle of perfume in a desert", depth=2)
prompt_3 = Prompt(prompt="Add sunset lighting to the perfume bottle scene", depth=3, suggestion_used="Try sunset colors", modified_suggestion = False)

# Manually attach the extra attributes
prompt_3.is_enhanced = True
prompt_3.enhanced_prompt = "A perfume bottle in a desert with glowing sunset hues"

# Save prompts through save_prompt() instead of insert_prompt()
datab.save_prompt(prompt_1, chat_id_1, user_id=1)
datab.save_prompt(prompt_2, chat_id_1, user_id=1)
datab.save_prompt(prompt_3, chat_id_1, user_id=1)

# Step 3: Generate and save images
# image_path_1 = "app/db/images/sauvage.png"
# image_path_2 = "app/db/images/sauvage_desert.png"
image_path_1 = "images/sauvage.png"
image_path_2 = "images/sauvage_desert.png"

image_1 = Image.open(image_path_1)
image_2 = Image.open(image_path_2)

prompt_image_1 = PromptImage(image=image_1, prompt_guidance=6.0, image_guidance=0.4,
                             input_prompt=prompt_1.id, output_prompt=prompt_1.id, save=True)
prompt_image_2 = PromptImage(image=image_2, prompt_guidance=7.0, image_guidance=0.5,
                             input_prompt=prompt_2.id, output_prompt=prompt_2.id, save=True)

datab.save_image(prompt_image_1, session_id=chat_id_1, user_id=1)
datab.save_image(prompt_image_2, session_id=chat_id_1, user_id=1)


# Adding data
# User 1: New Chat
datab.connect()
chat_id_3 = datab.insert_chat("User 1 - Extra Chat", 1)

extra_prompt_1 = Prompt(prompt="Design a futuristic drone", depth=1)
extra_prompt_2 = Prompt(prompt="Add neon lighting to the drone", depth=2)
extra_prompt_3 = Prompt(prompt="Place the drone in a cyberpunk city", depth=3)
extra_prompt_3.used_suggestion = True
extra_prompt_3.suggestion_used = "Try placing it in a city"
extra_prompt_3.modified_suggestion = True

datab.save_prompt(extra_prompt_1, chat_id_3, user_id=1)
datab.save_prompt(extra_prompt_2, chat_id_3, user_id=1)
datab.save_prompt(extra_prompt_3, chat_id_3, user_id=1)

image_3 = Image.open("images/test_image.jpg")
image_4 = Image.open("images/test_image2.jpg")
image_5 = Image.open("images/test_image3.jpg")


prompt_image_3 = PromptImage(image=image_3, prompt_guidance=5.5, image_guidance=0.3,
                             input_prompt=extra_prompt_1.id, output_prompt=extra_prompt_1.id, save=True)
prompt_image_4 = PromptImage(image=image_4, prompt_guidance=6.5, image_guidance=0.45,
                             input_prompt=extra_prompt_2.id, output_prompt=extra_prompt_2.id, save=True)
prompt_image_5 = PromptImage(image=image_5, prompt_guidance=6.5, image_guidance=0.45,
                             input_prompt=extra_prompt_3.id, output_prompt=extra_prompt_3.id, save=True)

datab.save_image(prompt_image_3, session_id=chat_id_3, user_id=1)
datab.save_image(prompt_image_4, session_id=chat_id_3, user_id=1)
datab.save_image(prompt_image_5, session_id=chat_id_3, user_id=1)

# User 2: Chat with Depth 5
datab.connect()
chat_id_4 = datab.insert_chat("User 2 - Long Chain", 2)

p1 = Prompt("A spaceship flying through clouds", depth=1)
p2 = Prompt("Add sunlight breaking through clouds", depth=2)
p3 = Prompt("Include a fleet of ships in the distance", depth=3)
p4 = Prompt("Make the ships look metallic", depth=4, suggestion_used="Make them metallic", modified_suggestion=False)
p5 = Prompt("Add reflections from the sun", depth=5)
p5.is_enhanced = True
p5.enhanced_prompt = "Fleet of metallic ships reflecting golden sunlight"

for p in [p1, p2, p3, p4, p5]:
    datab.save_prompt(p, chat_id_4, user_id=2)

img_paths = [
    "images/cat.png",
    "images/cat1.png",
    "images/cat2.png",
    "images/cat3.png",
    "images/cat4.png"
]

guidances = [(6.0, 0.4), (6.5, 0.45), (7.0, 0.5), (7.5, 0.55)]
prompts = [p1, p2, p3, p4, p5]

for path, (pg, ig), prompt in zip(img_paths, guidances, prompts):
    img = Image.open(path)
    pimg = PromptImage(image=img, prompt_guidance=pg, image_guidance=ig,
                       input_prompt=prompt.id, output_prompt=prompt.id, save=True)
    datab.save_image(pimg, session_id=chat_id_4, user_id=2)


datab.close()


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Calculating LPIPS for: images/dcf9213e-1f12-40d7-b40c-6f74b7b00f3a.png vs images/814db03f-4aeb-4785-82e1-4e54bdefc7c0.png
LPIPS result: 0.5469599962234497


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Calculating LPIPS for: images/74eb1497-3506-4ac6-8493-b3e7af4d21f5.png vs images/2b432ddc-6379-46cb-8714-00da658532c0.png
LPIPS result: 0.5211811065673828
Calculating LPIPS for: images/24d5f552-3eb7-43a6-91c9-b6fb8d2ac0d5.png vs images/74eb1497-3506-4ac6-8493-b3e7af4d21f5.png
LPIPS result: 0.6154640316963196


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

Calculating LPIPS for: images/73434783-d192-4401-8bee-e3ecb985975d.png vs images/383360af-711e-47a0-9f62-957ed5348088.png
LPIPS result: 0.21515347063541412
Calculating LPIPS for: images/ae19d1e5-632c-4e03-821c-76ec7b643147.png vs images/73434783-d192-4401-8bee-e3ecb985975d.png
LPIPS result: 0.7833430767059326
Calculating LPIPS for: images/ad3a19e0-77c7-4c50-a836-f7303b7f2eef.png vs images/ae19d1e5-632c-4e03-821c-76ec7b643147.png
LPIPS result: 0.29559803009033203


In [5]:
datab = Database()
datab.connect()

In [6]:
df_bert = datab.fetch_dataframe("SELECT * FROM bertscore_metrics")
df_guidance = datab.fetch_dataframe("SELECT * FROM guidance_metrics")
df_lpips = datab.fetch_dataframe("SELECT * FROM lpips_metrics")
df_func = datab.fetch_dataframe("SELECT * FROM functionality_metrics")
df_words = datab.fetch_dataframe("SELECT * FROM prompt_word_metrics")

print("BERTScore Metrics:\n", df_bert)
print("Guidance Metrics:\n", df_guidance)
print("LPIPS Metrics:\n", df_lpips)
print("Functionality Metrics:\n", df_func)
print("Prompt Word Metrics:\n", df_words)


BERTScore Metrics:
     id                             prompt_id  \
0    1  c65d7a06-2f85-4a78-b1ae-45cedcdfd18b   
1    2  0c2780ac-90c0-4592-8a76-bc4f9e750222   
2    3  dac60d93-9388-4225-aa9a-eb15b989ff3a   
3    4  2c0afd4f-9a5b-4720-bc78-a9b23da35ae3   
4    5  06fcabb6-18dd-4e3e-865a-36bed210d896   
5    6  493068b7-9bb9-49f8-9d60-044a275ce8c7   
6    7  4c89ea1a-0331-49bb-808d-cebbe293b767   
7    8  c2a9b619-85fe-4709-a5e2-8c8b80051ece   
8    9  7cb8fd21-9ebf-4ad0-affe-b3d0ec9f8682   
9   10  4c1cc055-bc8a-4219-92f2-7c247b671a79   
10  11  39d7b7b6-0d81-4b7e-a312-5d51c3d44cba   

                      previous_prompt_id  user_id  chat_id  depth  \
0                                   None        1        1      1   
1   c65d7a06-2f85-4a78-b1ae-45cedcdfd18b        1        1      2   
2   0c2780ac-90c0-4592-8a76-bc4f9e750222        1        1      3   
3                                   None        1        3      1   
4   2c0afd4f-9a5b-4720-bc78-a9b23da35ae3        1        3

### MMA Metrics Data Cheatsheet

Please note: two different users cannot have the same chat_id in your current database design.

```
CREATE TABLE IF NOT EXISTS chats (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    user_id INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
```

the id column (chat_id) is a unique, auto-incrementing primary key. Therefore, each chat_id is globally unique and tied to a single user_id. It's impossible for the same chat_id to belong to more than one user.

#### Setup

Make sure you have imported and connected:

```python
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

print(sys.path)
from db.database import Database
import uuid

db = Database()
db.connect()
```

---

#### Metric Overview

| Metric Type       | Table Name              | What It Measures                                    |
| ----------------- | ----------------------- | --------------------------------------------------- |
| **BERTScore**     | `bertscore_metrics`     | Novelty of current prompt vs previous               |
| **Guidance**      | `guidance_metrics`      | Prompt/image guidance values used during generation |
| **LPIPS**         | `lpips_metrics`         | Visual change between images                        |
| **Functionality** | `functionality_metrics` | Suggestion/enhancement usage stats                  |
| **Prompt Words**  | `prompt_word_metrics`   | Word-level analysis of prompts                      |

---

#### Fetch All Metrics

```python
df_bert = db.fetch_all_bertscore_metrics()
df_guidance = db.fetch_all_guidance_metrics()
df_lpips = db.fetch_all_lpips_metrics()
df_func = db.fetch_all_functionality_metrics()
df_words = db.fetch_all_prompt_word_metrics()
```

In [7]:
datab.fetch_all_bertscore_metrics()

,id,prompt_id,previous_prompt_id,user_id,chat_id,depth,bert_novelty
0,1,34ee7717-980a-4f84-8b00-b915464795dc,None,1,1,1,1.000000
1,2,8d76434a-8ad0-418a-84d5-f4128aa17209,34ee7717-980a-4f84-8b00-b915464795dc,1,1,2,0.040591
2,3,02d9f0c9-cc22-41e3-a21d-79897842a1ca,8d76434a-8ad0-418a-84d5-f4128aa17209,1,1,3,0.103445
3,4,ff138b0b-a20d-49b5-aae9-722fe8c4dad1,None,1,3,1,1.000000
4,5,50355bc4-3773-45c7-b14c-91d9a8f61ee6,ff138b0b-a20d-49b5-aae9-722fe8c4dad1,1,3,2,0.097047
5,6,4905610f-dcf9-45b8-88af-e3340bdbb328,50355bc4-3773-45c7-b14c-91d9a8f61ee6,1,3,3,0.123476
6,7,31d75dba-1aad-4388-ba29-2b307f4f4853,None,2,4,1,1.000000
7,8,de894b69-4e48-4674-82a8-1be14de23e16,31d75dba-1aad-4388-ba29-2b307f4f4853,2,4,2,0.090737
8,9,3e3cb1e7-ae15-4043-80cd-c8a12f50e5a1,de894b69-4e48-4674-82a8-1be14de23e16,2,4,3,0.145250
9,10,2be6620f-c44c-4381-b995-54bcd8e691fa,3e3cb1e7-ae15-4043-80cd-c8a12f50e5a1,2,4,4,0.153407


In [8]:
datab.fetch_all_guidance_metrics()

,id,prompt_id,user_id,chat_id,depth,prompt_guidance,image_guidance
0,1,34ee7717-980a-4f84-8b00-b915464795dc,1,1,1,6.0,0.40
1,2,8d76434a-8ad0-418a-84d5-f4128aa17209,1,1,2,7.0,0.50
2,3,ff138b0b-a20d-49b5-aae9-722fe8c4dad1,1,3,1,5.5,0.30
3,4,50355bc4-3773-45c7-b14c-91d9a8f61ee6,1,3,2,6.5,0.45
4,5,4905610f-dcf9-45b8-88af-e3340bdbb328,1,3,3,6.5,0.45
5,6,31d75dba-1aad-4388-ba29-2b307f4f4853,2,4,1,6.0,0.40
6,7,de894b69-4e48-4674-82a8-1be14de23e16,2,4,2,6.5,0.45
7,8,3e3cb1e7-ae15-4043-80cd-c8a12f50e5a1,2,4,3,7.0,0.50
8,9,2be6620f-c44c-4381-b995-54bcd8e691fa,2,4,4,7.5,0.55


In [9]:
datab.fetch_all_lpips_metrics()

,id,image_id,previous_image_id,user_id,chat_id,depth,lpips
0,1,97b9c312-ca71-40b6-86cb-6dc24b9fc3d4,None,1,1,1,NaN
1,2,8a0f2710-9dd1-4fc0-857d-b17d6c0a51cf,97b9c312-ca71-40b6-86cb-6dc24b9fc3d4,1,1,2,0.546960
2,3,fa5bf935-a1a5-44d8-b0eb-542c3efe45ed,None,1,3,1,NaN
3,4,b5221b44-286a-4b9b-81dd-0badc33f64f5,fa5bf935-a1a5-44d8-b0eb-542c3efe45ed,1,3,2,0.521181
4,5,789873ec-d9cf-4d05-8bfa-f9e13886daf5,b5221b44-286a-4b9b-81dd-0badc33f64f5,1,3,3,0.615464
5,6,8f8b1fb3-e1d0-4992-8ee9-78ce73f54888,None,2,4,1,NaN
6,7,d6c8110c-4c54-4e24-8d22-33b6748fd0b3,8f8b1fb3-e1d0-4992-8ee9-78ce73f54888,2,4,2,0.215153
7,8,ffc00f83-6675-4f5f-9b91-77553f5e45de,d6c8110c-4c54-4e24-8d22-33b6748fd0b3,2,4,3,0.783343
8,9,590a8c92-662b-42b6-83d0-cc65a9023816,ffc00f83-6675-4f5f-9b91-77553f5e45de,2,4,4,0.295598


In [10]:
from modules.metrics import calculate_lpips

score = calculate_lpips("images/cat1.png", "images/cat2.png")
print("Manual LPIPS:", score)

Calculating LPIPS for: images/cat1.png vs images/cat2.png
LPIPS result: 0.7833430767059326
Manual LPIPS: 0.7833430767059326


In [ ]:

datab.fetch_all_functionality_metrics()

In [ ]:

datab.fetch_all_prompt_word_metrics()

---

#### Fetch by User

```python
user_id = 1

df_bert = db.fetch_bertscore_by_user(user_id)
df_guidance = db.fetch_guidance_by_user(user_id)
df_lpips = db.fetch_lpips_by_user(user_id)
df_func = db.fetch_functionality_by_user(user_id)
df_words = db.fetch_prompt_word_metrics_by_user(user_id)
```

In [ ]:
user_id = 1

datab.fetch_bertscore_by_user(user_id)

In [ ]:
datab.fetch_guidance_by_user(user_id)

In [12]:
user_id = 1
datab.fetch_lpips_by_user(user_id)

,id,image_id,previous_image_id,user_id,chat_id,depth,lpips
0,1,97b9c312-ca71-40b6-86cb-6dc24b9fc3d4,None,1,1,1,NaN
1,2,8a0f2710-9dd1-4fc0-857d-b17d6c0a51cf,97b9c312-ca71-40b6-86cb-6dc24b9fc3d4,1,1,2,0.546960
2,3,fa5bf935-a1a5-44d8-b0eb-542c3efe45ed,None,1,3,1,NaN
3,4,b5221b44-286a-4b9b-81dd-0badc33f64f5,fa5bf935-a1a5-44d8-b0eb-542c3efe45ed,1,3,2,0.521181
4,5,789873ec-d9cf-4d05-8bfa-f9e13886daf5,b5221b44-286a-4b9b-81dd-0badc33f64f5,1,3,3,0.615464


In [ ]:
datab.fetch_functionality_by_user(user_id)

In [ ]:
datab.fetch_prompt_word_metrics_by_user(user_id)

---

#### Fetch by Chat

```python
chat_id = 2

df_bert = db.fetch_bertscore_by_chat(chat_id)
df_guidance = db.fetch_guidance_by_chat(chat_id)
df_lpips = db.fetch_lpips_by_chat(chat_id)
df_func = db.fetch_functionality_by_chat(chat_id)
df_words = db.fetch_prompt_word_metrics_by_chat(chat_id)
```



In [ ]:
chat_id = 1

datab.fetch_bertscore_by_chat(chat_id)

In [ ]:
datab.fetch_guidance_by_chat(chat_id)

In [17]:
chat_id = 1
datab.fetch_lpips_by_chat(chat_id)

,id,image_id,previous_image_id,user_id,chat_id,depth,lpips
0,1,97b9c312-ca71-40b6-86cb-6dc24b9fc3d4,None,1,1,1,NaN
1,2,8a0f2710-9dd1-4fc0-857d-b17d6c0a51cf,97b9c312-ca71-40b6-86cb-6dc24b9fc3d4,1,1,2,0.54696


In [ ]:
datab.fetch_functionality_by_chat(chat_id)

In [ ]:
datab.fetch_functionality_by_chat(3)

In [ ]:
datab.fetch_prompt_word_metrics_by_chat(chat_id)

---

#### Column Hints

##### `bertscore_metrics`

* `bert_novelty`: float (0–1)
* `prompt_id`, `previous_prompt_id`

##### `guidance_metrics`

* `prompt_guidance`, `image_guidance`: floats
* `depth`, `prompt_id`

##### `lpips_metrics`

* `lpips`: float (0–1, lower = more similar)
* `image_id`, `previous_image_id`

##### `functionality_metrics`

* `used_suggestion_pct`, `used_enhancement_pct`, `used_both_pct`, `no_ai_pct`: % as floats
* One row per (user, chat)

##### `prompt_word_metrics`

* `full_text`: full prompt
* `relevant_words`: comma-separated words
* `word_count`: number of words after stopword removal


#### Test all the fetch methods


In [4]:
def test_fetch_methods():
    db = Database()
    db.connect()

    print("\n========== USERS ==========")
    print("All users:", db.fetch_all_users().shape)
    print("User by ID (1):\n", db.fetch_user_by_id(1))
    print("User by Username:\n", db.fetch_user_by_username("second_test_user"))

    print("\n========== CHATS ==========")
    print("All chats:", db.fetch_all_chats().shape)
    print("Chats by User (1):\n", db.fetch_chats_by_user(1))
    print("Chat by ID (1):\n", db.fetch_chat_by_id(1))

    print("\n========== PROMPTS ==========")
    print("All prompts:", db.fetch_all_prompts().shape)
    print("Prompts by Chat (4):\n", db.fetch_prompts_by_chat(4))
    print("Prompts by User (1):\n", db.fetch_prompts_by_user(1))

    # Use a known prompt ID from your inserted test data
    print("Prompt by ID:\n", db.fetch_prompt_by_id("1"))  # Replace "1" with real prompt id if needed

    print("\n========== IMAGES ==========")
    print("All images:", db.fetch_all_images().shape)
    print("Images by User (1):\n", db.fetch_images_by_user(1))
    print("Images by Chat (3):\n", db.fetch_images_by_chat(3))
    print("Image by Path (images/sauvage.png):\n", db.fetch_image_by_path("images/sauvage.png"))

    # Replace with actual image_id from your test database
    sample_image_id = db.fetch_all_images().iloc[0]["id"]
    print(f"Image by ID ({sample_image_id}):\n", db.fetch_image_by_id(sample_image_id))

    print("\n========== METRICS ==========")

    print("All BERTScore metrics:\n", db.fetch_all_bertscore_metrics().shape)
    print("BERTScore by User (1):\n", db.fetch_bertscore_by_user(1))
    print("BERTScore by Chat (4):\n", db.fetch_bertscore_by_chat(4))

    print("\nGuidance metrics:", db.fetch_all_guidance_metrics().shape)
    print("Guidance by User (1):\n", db.fetch_guidance_by_user(1))

    print("\nLPIPS metrics:", db.fetch_all_lpips_metrics().shape)
    print("LPIPS by Chat (4):\n", db.fetch_lpips_by_chat(4))

    print("\nFunctionality metrics:", db.fetch_all_functionality_metrics().shape)
    print("Functionality by Chat (3):\n", db.fetch_functionality_by_chat(3))

    print("\nPrompt Word metrics:", db.fetch_all_prompt_word_metrics().shape)
    print("Prompt Word by Chat (1):\n", db.fetch_prompt_word_metrics_by_chat(1))

    db.close()

# Run the tests
test_fetch_methods()


========== USERS ==========
All users: (1, 3)
User by ID (1):
    id username password
0   1  vincent  vincent
User by Username:
 Empty DataFrame
Columns: [id, username, password]
Index: []

========== CHATS ==========
All chats: (2, 3)
Chats by User (1):
    id                     title  user_id
0   1  Chat 2025-06-19 19:10:41        1
1   2  Chat 2025-06-19 19:28:13        1
Chat by ID (1):
    id                     title  user_id
0   1  Chat 2025-06-19 19:10:41        1

========== PROMPTS ==========
All prompts: (5, 12)
Prompts by Chat (4):
 Empty DataFrame
Columns: [id, chat_id, user_id, prompt, depth, used_suggestion, modified_suggestion, suggestion_used, is_enhanced, enhanced_prompt, image_in_id, images_out]
Index: []
Prompts by User (1):
                                      id  chat_id  user_id  \
0  c4cfa219-4974-4b62-add7-00ab81a23d34        1        1   
1  c33f26da-c4f4-44bd-8f76-613e9295e66a        1        1   
2  a2aefbb6-a9f2-44d5-b5a8-459c77c6858c        1        1 

DatabaseError: Execution failed on sql 'SELECT * FROM bertscore_metrics': no such table: bertscore_metrics

#### stuff for testing the inserting (insert_*)

In [ ]:
datab = Database()
datab.connect()

In [ ]:
# Clears all previous database entries
datab.reset_tables()

In [ ]:
# --- Step 1: Create Users ---
datab.insert_user("first_test_user", "password123")
datab.insert_user("second_test_user", "password456")

In [ ]:
# --- Step 2: Create Chats ---
datab.insert_chat("First Test Chat", 1)
datab.insert_chat("Second Test Chat", 2)

chat_id_1=1
chat_id_2=2

# --- Step 3: Create Prompts ---
prompt_id_0 = str(uuid.uuid4())
prompt_id_1 = str(uuid.uuid4())
prompt_id_2 = str(uuid.uuid4())
prompt_id_3 = str(uuid.uuid4())
prompt_id_4 = str(uuid.uuid4())

# Inputs for insert_prompt function:
# prompt_id, chat_id, user_id, prompt, depth, used_suggestion=False, modified_suggestion=False, suggestion_used=None, is_enhanced=False, enhanced_prompt=None, image_in_id=None, images_out=None

# note: as it stands Gilian's design expects each prompt_id to be a globally unique ID, such as a UUID.
# this is because the code says "CREATE TABLE IF NOT EXISTS prompts (id TEXT PRIMARY KEY,...)"
# we can maybe create a composite key (e.g., PRIMARY KEY (chat_id, id)) so that we distinguish them by chat/user
datab.insert_prompt(
    id=prompt_id_0, chat_id=chat_id_1, user_id=1,
    prompt="Generate a bottle of perfume", depth=1
)

datab.insert_prompt(
    id=prompt_id_1, chat_id=chat_id_1, user_id=1,
    prompt="Generate a bottle of perfume in a desert", depth=2
)

datab.insert_prompt(
    id=prompt_id_2, chat_id=chat_id_1, user_id=1,
    prompt="Add sunset lighting to the perfume bottle scene", depth=3,
    used_suggestion=True, suggestion_used="Try sunset colors", is_enhanced=True,
    enhanced_prompt="A perfume bottle in a desert with glowing sunset hues"
)

datab.insert_prompt(
    id=prompt_id_3, chat_id=chat_id_2, user_id=2,
    prompt="Design a stylish water bottle sticker", depth=1,
    used_suggestion=True, modified_suggestion=True, suggestion_used="Urban aesthetic",
    is_enhanced=True, enhanced_prompt="Water bottle sticker with urban graffiti"
)

datab.insert_prompt(
    id=prompt_id_4, chat_id=chat_id_2, user_id=2,
    prompt="Make it into a metallic sticker", depth=2,
    is_enhanced=False
)

# As can be seen in the code below, the input_prompt_id cannot be a str
# if not isinstance(input_prompt_id, str):
#     raise ValueError("input_prompt_id must be an integer")

# --- Step 4: Create Images ---
image_id_1 = str(uuid.uuid4())
image_id_2 = str(uuid.uuid4())
image_id_3 = str(uuid.uuid4())
image_id_4 = str(uuid.uuid4())

# Inputs for insert_image function:
# image_id: str, user_id: int, chat_id: int, prompt_guidance: float, image_guidance: float, path: str, input_prompt_id: str, output_prompt_id: str,
datab.insert_image(
    id=image_id_1, user_id=1, chat_id=chat_id_1,
    prompt_guidance=6.0, image_guidance=0.4,
    path="app/db/images/sauvage.png",
    input_prompt_id=prompt_id_1, output_prompt_id=prompt_id_1
)

datab.insert_image(
    id=image_id_2, user_id=1, chat_id=chat_id_1,
    prompt_guidance=7.0, image_guidance=0.5,
    path="app/db/images/sauvage_desert.png",
    input_prompt_id=prompt_id_2, output_prompt_id=prompt_id_2
)

datab.insert_image(
    id=image_id_3, user_id=2, chat_id=chat_id_2,
    prompt_guidance=5.5, image_guidance=0.3,
    path="app/db/images/test_image2.jpg",
    input_prompt_id=prompt_id_3, output_prompt_id=prompt_id_3
)

datab.insert_image(
    id=image_id_4, user_id=2, chat_id=chat_id_2,
    prompt_guidance=6.2, image_guidance=0.2,
    path="app/db/images/test_image4.jpg",
    input_prompt_id=prompt_id_4, output_prompt_id=prompt_id_4
)

In [ ]:
with Database() as data_base:  # adjust path if needed
    print("=== USERS ===")
    print(data_base.fetch_all_users())
    
    print("=== CHATS ===")
    print(data_base.fetch_all_chats())
    
    print("=== PROMPTS ===")
    print(data_base.fetch_all_prompts())
    
    print("=== IMAGES ===")
    print(data_base.fetch_all_images())

In [ ]:
data_base = Database()
data_base.connect()

In [ ]:
current_prompt_df = data_base.fetch_prompt_by_id('8dd6a2f4-6622-4a63-a093-8eac09b07b0b', pandas=True)

In [ ]:
if not current_prompt_df.empty:
    depth = current_prompt_df.iloc[0]["depth"]
else:
    depth = 0

print(depth)

In [ ]:
prior_images = data_base.fetch_images_by_chat(chat_id_1, pandas=True)
# print(f"prior_images1: {prior_images}")
prior_images = prior_images.merge(data_base.fetch_all_prompts(), how="left", left_on="output_prompt_id", right_on="id")
# print(f"prior_images2: {prior_images}")
prior_images = prior_images[prior_images["depth"] == depth - 1]
print(f"prior_images3: {prior_images}")

In [ ]:
previous_image_id = None
previous_image_path = None
if not prior_images.empty:
        previous_image_id = prior_images.iloc[0]["id_x"]  # id_x = image.id
        print(previous_image_id)
        previous_image_path = prior_images.iloc[0]["path"]
        print(previous_image_path)


In [ ]:
current_images = data_base.fetch_images_by_chat(chat_id_1, pandas=True)
# print(f"prior_images1: {prior_images}")
current_images = current_images.merge(data_base.fetch_all_prompts(), how="left", left_on="output_prompt_id", right_on="id")
current_images = current_images[current_images["depth"] == depth]
print(f"current_images3: {current_images}")
print(current_images.iloc[0]["id_x"])
print(current_images.iloc[0]["path"])

In [ ]:
from modules.metrics import get_or_compute_bertscore, extract_relevant_words, get_or_compute_lpips

In [ ]:
lpips_score = None
if previous_image_path:
    lpips_score = get_or_compute_lpips("6122bcfc-fb9d-4f2c-afdf-bd93192b4b22", "images/sauvage_desert.png", "images/sauvage.png")
else:
    lpips_score = None

print(str(lpips_score))

In [ ]:
ls

In [ ]:
data_base = Database()
data_base.connect()

In [ ]:

datab.fetch_all_prompts()

In [ ]:
# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

datab.fetch_all_users(pandas=True)

In [ ]:

# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

datab.fetch_all_images(pandas=True)

In [ ]:
datab.close()